# 17장 딥러닝을 이용한 자연어 처리

[<img src="https://raw.githubusercontent.com/taehojo/taehojo.github.io/master/assets/images/linktocolab.png" align="left"/> ](https://colab.research.google.com/github/taehojo/deeplearning/blob/master/colab/ch17-colab.ipynb)

## 1. 텍스트의 토큰화

In [3]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten,Embedding
from tensorflow.keras.utils import to_categorical
from numpy import array

# 케라스의 텍스트 전처리와 관련한 함수중 text_to_word_sequence 함수를 불러옵니다.
from tensorflow.keras.preprocessing.text import text_to_word_sequence
 

## 2.텍스트를 읽고 토큰 생성

In [4]:
# 텍스트 리뷰 자료를 지정합니다.
docs = ["너무 재밌네요","최고예요","참 잘 만든 영화예요","추천하고 싶은 영화입니다","한번 더 보고싶네요","글쎄요","별로예요","생각보다 지루하네요","연기가 어색해요","재미없어요"]

# 긍정 리뷰는 1, 부정 리뷰는 0으로 클래스를 지정합니다.
classes = array([1,1,1,1,1,0,0,0,0,0])

# 토큰화 
token = Tokenizer()
token.fit_on_texts(docs)
print(token.word_index)


{'너무': 1, '재밌네요': 2, '최고예요': 3, '참': 4, '잘': 5, '만든': 6, '영화예요': 7, '추천하고': 8, '싶은': 9, '영화입니다': 10, '한번': 11, '더': 12, '보고싶네요': 13, '글쎄요': 14, '별로예요': 15, '생각보다': 16, '지루하네요': 17, '연기가': 18, '어색해요': 19, '재미없어요': 20}


# 3. 토큰 저장 (추후 예측에 활용)

In [7]:
# 토큰 저장 saving
import pickle


tokenpath="./data/model/ch17_review_token.pickle"

with open(tokenpath, 'wb') as handle:
    pickle.dump(token, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [8]:
# 토큰화 결과 확인
x = token.texts_to_sequences(docs)
print("\n리뷰 텍스트, 토큰화 결과:\n",  x)


리뷰 텍스트, 토큰화 결과:
 [[1, 2], [3], [4, 5, 6, 7], [8, 9, 10], [11, 12, 13], [14], [15], [16, 17], [18, 19], [20]]


In [9]:
# 패딩, 서로 다른 길이의 데이터를 4로 맞추어 줍니다.
padded_x = pad_sequences(x, 4)  
print("\n패딩 결과:\n", padded_x)


패딩 결과:
 [[ 0  0  1  2]
 [ 0  0  0  3]
 [ 4  5  6  7]
 [ 0  8  9 10]
 [ 0 11 12 13]
 [ 0  0  0 14]
 [ 0  0  0 15]
 [ 0  0 16 17]
 [ 0  0 18 19]
 [ 0  0  0 20]]


In [10]:
# 모델 체크(최적 학습), 종료
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
import os

# 모델 최적화를 위한 설정구간입니다.
MODEL_DIR = './data/model/'
if not os.path.exists(MODEL_DIR):
    os.mkdir(MODEL_DIR)

modelpath="./data/model/ch17_review_md.hdf5"
checkpointer = ModelCheckpoint(filepath=modelpath, monitor='accuracy', verbose=1, save_best_only=True)
early_stopping_callback = EarlyStopping(monitor='accuracy', patience=10)



In [11]:
# 임베딩에 입력될 단어의 수를 지정합니다.
word_size = len(token.word_index) +1

# 단어 임베딩을 포함하여 딥러닝 모델을 만들고 결과를 출력합니다.
model = Sequential()
model.add(Embedding(word_size, 8, input_length=4))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 4, 8)              168       
_________________________________________________________________
flatten (Flatten)            (None, 32)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 33        
Total params: 201
Trainable params: 201
Non-trainable params: 0
_________________________________________________________________


2022-06-10 16:58:00.350861: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [12]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(padded_x, classes, epochs=40,callbacks=[early_stopping_callback,checkpointer])


Epoch 1/40
1/1 [==============================] - 0s 321ms/step - loss: 0.6879 - accuracy: 0.5000

Epoch 00001: accuracy improved from -inf to 0.50000, saving model to ./data/model/ch17_review_md.hdf5
Epoch 2/40
1/1 [==============================] - 0s 2ms/step - loss: 0.6855 - accuracy: 0.5000

Epoch 00002: accuracy did not improve from 0.50000
Epoch 3/40
1/1 [==============================] - 0s 2ms/step - loss: 0.6830 - accuracy: 0.6000

Epoch 00003: accuracy improved from 0.50000 to 0.60000, saving model to ./data/model/ch17_review_md.hdf5
Epoch 4/40
1/1 [==============================] - 0s 2ms/step - loss: 0.6806 - accuracy: 0.6000

Epoch 00004: accuracy did not improve from 0.60000
Epoch 5/40
1/1 [==============================] - 0s 2ms/step - loss: 0.6782 - accuracy: 0.8000

Epoch 00005: accuracy improved from 0.60000 to 0.80000, saving model to ./data/model/ch17_review_md.hdf5
Epoch 6/40
1/1 [==============================] - 0s 2ms/step - loss: 0.6757 - accuracy: 0.8000

Ep

In [13]:
# 예측률 검증
#  "참 너무 별로예요" 는 부정이지만, 분석 데이터 기준으로는 긍정 평가임
test_doc = ["글쎄요 너무 최고예요", "참 너무 별로예요", "글쎄요 지루하네요","재미없어요","이상한 문구는 어떻게 되나요"];
classes_y = array([1,0,0,0,0])

test = token.texts_to_sequences(test_doc)
print(test)
padded_test = pad_sequences(test, 4)  
print(padded_test)

#model.predict(padded_test)

print("\n Accuracy: %.4f" % (model.evaluate(padded_test, classes_y)[1]))

[[14, 1, 3], [4, 1, 15], [14, 17], [20], []]
[[ 0 14  1  3]
 [ 0  4  1 15]
 [ 0  0 14 17]
 [ 0  0  0 20]
 [ 0  0  0  0]]
1/1 [==============================] - 0s 103ms/step - loss: 0.6609 - accuracy: 1.0000

 Accuracy: 1.0000
